deep-graspDetection_v0.1

A source code developed by Shehan Caldera (shehancaldera@gmail.com) for the purpose of training the robot grasp
detection convolutional neural network (RGD-CNN) on the augmented Cornell Grasp data. The training data has the
following format ==> (RGB image, grasp rectangle params : x, y, theta, h, w ).

@SAMPLE_NMBR is 9000 from the augmented dataset. Real-time data augmentation is employed using keras ImagePreprocessing
class. The model has a convolutional base with the architecture of Inception-ResNet

In [4]:
# Import_keras
import keras
from keras import backend as K

keras.__version__
K.clear_session() # Init resource clean-up & refresh

In [6]:
# Define the imports
import os
import numpy as np
import math
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean, rotate
from PIL import Image
from PIL import ImageOps
import scipy as sp
from skimage.util import crop
import matplotlib.pyplot as plt
import numpy as np
import pypcd
import time
from keras import backend as K
from keras.applications import Xception, VGG16, ResNet50, InceptionResNetV2
from keras import models
from keras import layers
from keras.models import Sequential
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from shapely.geometry import Polygon
import math

# Global Variables
BASE_DIR = '/media/baxter/DataDisk/Cornell Grasps Dataset/rgb_aug'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')
TARGET_IMAGE_WIDTH = 224
SAMPLE_NUMBR = 9000  # Number of sub-samples to select from the data: image, label

In [16]:
# Function for reading CGD data into network variables
def read_cgd_data(sample_number):
    x_train = []
    y_train = []
    
    x_valid = []
    y_valid = []
    
    print(time.strftime("%d.%m.%Y_%H.%M.%S", time.localtime()) + ': Reading the CGD data...')

    for i in range(0, 49359, int(49359 / sample_number)):
        img_filename = os.path.join(TRAIN_DIR, "img" + str(i) + "." + 'png')
        img = Image.open(img_filename)
        # resized_img = img.resize((TARGET_IMAGE_WIDTH, TARGET_IMAGE_WIDTH), resample=Image.NEAREST)

        img_array = np.asarray(img, dtype='float32')
        x_train.append(img_array)

        grasp_filename = os.path.join(TRAIN_DIR, "grasp" + str(i) + "." + 'txt')
        grasp = np.loadtxt(grasp_filename, dtype='float32')
        y_train.append(grasp)
        print(time.strftime("%Y-%m-%d %H.%M.%S", time.localtime()) + ': Training datapoint # loaded:' + str(i))

    x_train = np.asarray(x_train, dtype='float32')
    y_train = np.asarray(y_train, dtype='float32')
    
    return x_train, y_train

In [18]:
def network_model():
    # Import ResNet50 with imagenet weights
    conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(TARGET_IMAGE_WIDTH, TARGET_IMAGE_WIDTH, 3))

    # Initiate the sequential model
    model = Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(5, activation='relu'))  # Last layer does not have an activation as this is a vector regression

    # Freezing the weights of the VGG16 on imagenet weights
    conv_base.trainable = False

    # Verifying the model architecture by inspecting its summary
    model.summary()

    # Compile with Adam optimizer
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='mse', metrics=['acc'])

    return model

In [ ]:
from keras.callbacks import TensorBoard

# Training with realtime data augmentation with ImagePreprocessing class
datagen = ImageDataGenerator(rotation_range=0.5,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             validation_split=0.2)

# Read the data
(x_train, y_train) = read_cgd_data(SAMPLE_NUMBR)

# Model definition & raining params
model = network_model()
epochs = 30
batch_size = 32

# Create callbacks for Tensorboard
tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()))

# Fits the model on batches with real-time data augmentation
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, subset="training"),
                              steps_per_epoch=len(x_train) / batch_size, epochs=epochs,
                              callbacks=[tensorboard],
                              validation_data=(datagen.flow(x_train, y_train, batch_size=batch_size, subset="validation")),
                              #validation_steps=len(x_train)*0.2 / batch_size,
                              verbose=1)

18.05.2018_15.22.58: Reading the CGD data...


2018-05-18 15.22.58: Training datapoint # loaded:0
2018-05-18 15.22.58: Training datapoint # loaded:5
2018-05-18 15.22.58: Training datapoint # loaded:10
2018-05-18 15.22.58: Training datapoint # loaded:15
2018-05-18 15.22.58: Training datapoint # loaded:20
2018-05-18 15.22.58: Training datapoint # loaded:25


2018-05-18 15.22.58: Training datapoint # loaded:30
2018-05-18 15.22.58: Training datapoint # loaded:35
2018-05-18 15.22.58: Training datapoint # loaded:40
2018-05-18 15.22.58: Training datapoint # loaded:45
2018-05-18 15.22.58: Training datapoint # loaded:50
2018-05-18 15.22.58: Training datapoint # loaded:55
2018-05-18 15.22.58: Training datapoint # loaded:60
2018-05-18 15.22.58: Training datapoint # loaded:65
2018-05-18 15.22.58: Training datapoint # loaded:70


2018-05-18 15.22.58: Training datapoint # loaded:75
2018-05-18 15.22.58: Training datapoint # loaded:80
2018-05-18 15.22.58: Training datapoint # loaded:85
2018-05-18 15.22.58: Training datapoint # loaded:90
2018-05-18 15.22.58: Training datapoint # loaded:95
2018-05-18 15.22.58: Training datapoint # loaded:100
2018-05-18 15.22.58: Training datapoint # loaded:105
2018-05-18 15.22.58: Training datapoint # loaded:110
2018-05-18 15.22.58: Training datapoint # loaded:115
2018-05-18 15.22.58: Training datapoint # loaded:120
2018-05-18 15.22.58: Training datapoint # loaded:125


2018-05-18 15.22.58: Training datapoint # loaded:130
2018-05-18 15.22.59: Training datapoint # loaded:135
2018-05-18 15.22.59: Training datapoint # loaded:140
2018-05-18 15.22.59: Training datapoint # loaded:145
2018-05-18 15.22.59: Training datapoint # loaded:150
2018-05-18 15.22.59: Training datapoint # loaded:155
2018-05-18 15.22.59: Training datapoint # loaded:160
2018-05-18 15.22.59: Training datapoint # loaded:165
2018-05-18 15.22.59: Training datapoint # loaded:170
2018-05-18 15.22.59: Training datapoint # loaded:175
2018-05-18 15.22.59: Training datapoint # loaded:180
2018-05-18 15.22.59: Training datapoint # loaded:185
2018-05-18 15.22.59: Training datapoint # loaded:190
2018-05-18 15.22.59: Training datapoint # loaded:195
2018-05-18 15.22.59: Training datapoint # loaded:200


...

2018-05-18 15.24.11: Training datapoint # loaded:48845
2018-05-18 15.24.11: Training datapoint # loaded:48850
2018-05-18 15.24.11: Training datapoint # loaded:48855
2018-05-18 15.24.11: Training datapoint # loaded:48860
2018-05-18 15.24.11: Training datapoint # loaded:48865
2018-05-18 15.24.11: Training datapoint # loaded:48870
2018-05-18 15.24.11: Training datapoint # loaded:48875
2018-05-18 15.24.11: Training datapoint # loaded:48880
2018-05-18 15.24.11: Training datapoint # loaded:48885
2018-05-18 15.24.11: Training datapoint # loaded:48890
2018-05-18 15.24.11: Training datapoint # loaded:48895
2018-05-18 15.24.11: Training datapoint # loaded:48900
2018-05-18 15.24.11: Training datapoint # loaded:48905
2018-05-18 15.24.11: Training datapoint # loaded:48910
2018-05-18 15.24.11: Training datapoint # loaded:48915
2018-05-18 15.24.11: Training datapoint # loaded:48920
2018-05-18 15.24.11: Training datapoint # loaded:48925
2018-05-18 15.24.11: Training datapoint # loaded:48930
2018-05-18

2018-05-18 15.24.11: Training datapoint # loaded:49170
2018-05-18 15.24.11: Training datapoint # loaded:49175
2018-05-18 15.24.11: Training datapoint # loaded:49180
2018-05-18 15.24.11: Training datapoint # loaded:49185
2018-05-18 15.24.11: Training datapoint # loaded:49190
2018-05-18 15.24.11: Training datapoint # loaded:49195
2018-05-18 15.24.11: Training datapoint # loaded:49200
2018-05-18 15.24.11: Training datapoint # loaded:49205
2018-05-18 15.24.11: Training datapoint # loaded:49210
2018-05-18 15.24.11: Training datapoint # loaded:49215
2018-05-18 15.24.11: Training datapoint # loaded:49220
2018-05-18 15.24.11: Training datapoint # loaded:49225
2018-05-18 15.24.11: Training datapoint # loaded:49230
2018-05-18 15.24.11: Training datapoint # loaded:49235
2018-05-18 15.24.11: Training datapoint # loaded:49240
2018-05-18 15.24.11: Training datapoint # loaded:49245
2018-05-18 15.24.11: Training datapoint # loaded:49250
2018-05-18 15.24.11: Training datapoint # loaded:49255
2018-05-18

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2565      
Total params: 24,639,365
Trainable params: 1,051,653
Non-trainable params: 23,587,712
_________________________________________________________________


Instructions for updating:
Use the retry module or similar alternatives.


TypeError: unsupported operand type(s) for +: 'TensorBoard' and 'list'